##**Laptops data case study**

In [ ]:
use phpmyadmin;

SELECT * FROM laptop
WHERE Company IS NULL AND TypeName IS NULL AND  ScreenResolution IS NULL AND Cpu IS NULL AND Ram IS NULL
AND Memory IS NULL AND Gpu IS NULL AND OpSys IS NULL AND
WEIGHT IS NULL ;


ALTER TABLE laptop
CHANGE `Unnamed: 0` `index` VARCHAR(255);


SET SQL_SAFE_UPDATES = 0;

delete from laptop where Inches in (select Inches from laptop
where Inches =0 and Company !="Dell");

ALTER TABLE laptop MODIFY COLUMN Inches DECIMAL(10,1);

update  laptop l1
set Ram= (select replace(Ram, "GB","") FROM laptop l2 where l2.index= l1.index);

alter table laptop modify column Ram INTEGER;
select * from laptop;
update laptop l1
set Weight =(
select replace(Weight,"kg","") from laptop l2 where l2.index=l1.index);
select * from laptop;
alter table laptop modify column Weight decimal(10,2);


alter table laptop
add column cpu_brand varchar(255) after Cpu,
add column cpu_name Varchar(255) after cpu_brand,
add column cpu_speed Decimal(10,1) after cpu_name;

update laptop l1
set cpu_brand= (select substring_index( Cpu," ",1) from laptop l2 where l2.index=l1.index);
update   laptop l1
set cpu_speed=
(select replace(substring_index(Cpu," " ,-1),"GHz","") from laptop l2 where l1.index=l2.index);

update laptop l1
set cpu_name=(select substring_index(substring_index( Cpu," ",-2)," ", 1) from laptop l2 where l1.index= l2.index);

select * from laptop;
update laptop l1
set cpu_name=(
select   replace(replace(cpu,substring_index(replace( cpu,cpu_brand, "")," ", -1),""),cpu_brand,"") from laptop l2  where l1.index=l2.index);

alter table laptop drop column Cpu;



update laptop l1
set Opsys=(
select lower(Opsys) from laptop l2 where l1.index =l2.index);

update laptop
set Opsys=
CASE
when opsys like "%mac%"  then "macOS"
when opsys like "%windows%" then "Windows"
when opsys like "%lin%" then "Linux"
when opsys like "%no os" then "N/A"
else "others"
end;
alter table laptop
add column storage INTEGER after Memory,
ADD column memory_type VARCHAR(255) after storage,
ADD COLUMN primary_storage INTEGER AFTER memory_type,
ADD COLUMN secondary_storage INTEGER AFTER primary_storage;

ALTER TABLE laptop modify column Price integer;
use phpmyadmin;


select Memory, substring_index(memory," ",1) from laptop;
alter table laptop drop column secondary_storage;
alter table laptop add column screen_length integer after ScreenResolution, add column screen_width integer after screen_length,
add column touchscreen integer after screen_width;
select * from laptop;
select ScreenResolution, substring_index(substring_index(ScreenResolution," ",-1),"x",1) from laptop;

update laptop
set screen_length=substring_index(substring_index(ScreenResolution," ",-1),"x",1),
 screen_width=substring_index(substring_index(ScreenResolution," ",-1),"x",-1);

 select ScreenResolution,
 Case
 when ScreenResolution like "%Touchscreen%" then 1 else 0 end as screen from laptop;
 update laptop
set touchscreen= Case
 when ScreenResolution like "%Touchscreen%" then 1 else 0 end;

alter table laptop drop column screenResolution;

select memory, REGEXP_SUBSTR(Memory,'[0-9]+'),
case when memory like "%+%" then
REGEXP_SUBSTR(substring_index(memory,"+",-1), '[0-9]+') else 0 end as se from laptop;

update laptop
set primary_storage=REGEXP_SUBSTR(Memory,'[0-9]+'),
secondary_storage= case when memory like "%+%" then
REGEXP_SUBSTR(substring_index(memory,"+",-1), '[0-9]+') else 0 end;

select memory,
CASE
	WHEN Memory LIKE '%SSD%' AND Memory LIKE '%HDD%'  or memory like "%+%" THEN 'Hybrid'
    WHEN Memory LIKE '%SSD%' THEN 'SSD'
    WHEN Memory LIKE '%HDD%' THEN 'HDD'
    WHEN Memory LIKE '%Flash Storage%' THEN 'Flash Storage'
    WHEN Memory LIKE '%Hybrid%' THEN 'Hybrid'

    WHEN Memory LIKE '%Flash Storage%' AND Memory LIKE '%HDD%' THEN 'Hybrid'
    ELSE NULL
END AS 'memory_type'
FROM laptop;

update laptop
set memory_type= CASE
	WHEN Memory LIKE '%SSD%' AND Memory LIKE '%HDD%'  or memory like "%+%" THEN 'Hybrid'
    WHEN Memory LIKE '%SSD%' THEN 'SSD'
    WHEN Memory LIKE '%HDD%' THEN 'HDD'
    WHEN Memory LIKE '%Flash Storage%' THEN 'Flash Storage'
    WHEN Memory LIKE '%Hybrid%' THEN 'Hybrid'

    WHEN Memory LIKE '%Flash Storage%' AND Memory LIKE '%HDD%' THEN 'Hybrid'
    ELSE NULL
END;
case when primary_storage <=2 then primary_storage*1024 else primary_storage end as p
 FROM phpmyadmin.laptop;
 use phpmyadmin;
 update laptop
 set primary_storage =case when primary_storage <=2 then primary_storage*1024 else primary_storage end ,
 secondary_storage=case when secondary_storage <=2 then secondary_storage*1024 else secondary_storage end;

 select * from laptop;

alter table laptop
drop column storage;
select Gpu, substring_index(Gpu," ",1) from laptop;
update laptop
set Gpu=substring_index(Gpu," ",1);
select Company,
sum(case when touchscreen =1 then 1 else 0 end) as yes,
sum(case when touchscreen =1 then 0 else 1 end) as no
 from laptop
group by Company
order by yes desc




In [ ]:
SELECT * FROM sql_cx_live.laptops;

SELECT * FROM laptops;

CREATE TABLE laptops_backup LIKE laptops;

INSERT INTO laptops_backup
SELECT * FROM laptops;

SELECT DATA_LENGTH/1024 FROM information_schema.TABLES
WHERE TABLE_SCHEMA = 'sql_cx_live'
AND TABLE_NAME = 'laptops';

SELECT * FROM laptops;

ALTER TABLE laptops DROP COLUMN `Unnamed: 0`;

SELECT * FROM laptops;

DELETE FROM laptops
WHERE `index` IN (SELECT `index` FROM laptops
WHERE Company IS NULL AND TypeName IS NULL AND Inches IS NULL
AND ScreenResolution IS NULL AND Cpu IS NULL AND Ram IS NULL
AND Memory IS NULL AND Gpu IS NULL AND OpSys IS NULL AND
WEIGHT IS NULL AND Price IS NULL);

ALTER TABLE laptops MODIFY COLUMN Inches DECIMAL(10,1);

SELECT * FROM laptops;

UPDATE laptops l1
SET Ram = (SELECT REPLACE(Ram,'GB','') FROM laptops l2 WHERE l2.index = l1.index);

SELECT * FROM laptops;

ALTER TABLE laptops MODIFY COLUMN Ram INTEGER;

SELECT DATA_LENGTH/1024 FROM information_schema.TABLES
WHERE TABLE_SCHEMA = 'sql_cx_live'
AND TABLE_NAME = 'laptops';

UPDATE laptops l1
SET Weight = (SELECT REPLACE(Weight,'kg','')
		   FROM laptops l2 WHERE l2.index = l1.index);

SELECT * FROM laptops;

UPDATE laptops l1
SET Price = (SELECT ROUND(Price)
			FROM laptops l2 WHERE l2.index = l1.index);

ALTER TABLE laptops MODIFY COLUMN Price INTEGER;

SELECT DISTINCT OpSys FROM laptops;

-- mac
-- windows
-- linux
-- no os
-- Android chrome(others)

SELECT OpSys,
CASE
	WHEN OpSys LIKE '%mac%' THEN 'macos'
    WHEN OpSys LIKE 'windows%' THEN 'windows'
    WHEN OpSys LIKE '%linux%' THEN 'linux'
    WHEN OpSys = 'No OS' THEN 'N/A'
    ELSE 'other'
END AS 'os_brand'
FROM laptops;

UPDATE laptops
SET OpSys =
CASE
	WHEN OpSys LIKE '%mac%' THEN 'macos'
    WHEN OpSys LIKE 'windows%' THEN 'windows'
    WHEN OpSys LIKE '%linux%' THEN 'linux'
    WHEN OpSys = 'No OS' THEN 'N/A'
    ELSE 'other'
END;

SELECT * FROM laptops;

ALTER TABLE laptops
ADD COLUMN gpu_brand VARCHAR(255) AFTER Gpu,
ADD COLUMN gpu_name VARCHAR(255) AFTER gpu_brand;

SELECT * FROM laptops;

UPDATE laptops l1
SET gpu_brand = (SELECT SUBSTRING_INDEX(Gpu,' ',1)
				FROM laptops l2 WHERE l2.index = l1.index);

UPDATE laptops l1
SET gpu_name = (SELECT REPLACE(Gpu,gpu_brand,'')
				FROM laptops l2 WHERE l2.index = l1.index);

SELECT * FROM laptops;

ALTER TABLE laptops DROP COLUMN Gpu;

SELECT * FROM laptops;

ALTER TABLE laptops
ADD COLUMN cpu_brand VARCHAR(255) AFTER Cpu,
ADD COLUMN cpu_name VARCHAR(255) AFTER cpu_brand,
ADD COLUMN cpu_speed DECIMAL(10,1) AFTER cpu_name;

SELECT * FROM laptops;

UPDATE laptops l1
SET cpu_brand = (SELECT SUBSTRING_INDEX(Cpu,' ',1)
				 FROM laptops l2 WHERE l2.index = l1.index);

UPDATE laptops l1
SET cpu_speed = (SELECT CAST(REPLACE(SUBSTRING_INDEX(Cpu,' ',-1),'GHz','')
				AS DECIMAL(10,2)) FROM laptops l2
                WHERE l2.index = l1.index);

UPDATE laptops l1
SET cpu_name = (SELECT
					REPLACE(REPLACE(Cpu,cpu_brand,''),SUBSTRING_INDEX(REPLACE(Cpu,cpu_brand,''),' ',-1),'')
					FROM laptops l2
					WHERE l2.index = l1.index);

SELECT * FROM laptops;

ALTER TABLE laptops DROP COLUMN Cpu;

SELECT ScreenResolution,
SUBSTRING_INDEX(SUBSTRING_INDEX(ScreenResolution,' ',-1),'x',1),
SUBSTRING_INDEX(SUBSTRING_INDEX(ScreenResolution,' ',-1),'x',-1)
FROM laptops;

ALTER TABLE laptops
ADD COLUMN resolution_width INTEGER AFTER ScreenResolution,
ADD COLUMN resolution_height INTEGER AFTER resolution_width;

SELECT * FROM laptops;

UPDATE laptops
SET resolution_width = SUBSTRING_INDEX(SUBSTRING_INDEX(ScreenResolution,' ',-1),'x',1),
resolution_height = SUBSTRING_INDEX(SUBSTRING_INDEX(ScreenResolution,' ',-1),'x',-1);



ALTER TABLE laptops
ADD COLUMN touchscreen INTEGER AFTER resolution_height;

SELECT ScreenResolution LIKE '%Touch%' FROM laptops;

UPDATE laptops
SET touchscreen = ScreenResolution LIKE '%Touch%';

SELECT * FROM laptops;

ALTER TABLE laptops
DROP COLUMN ScreenResolution;

SELECT * FROM laptops;

SELECT cpu_name,
SUBSTRING_INDEX(TRIM(cpu_name),' ',2)
FROM laptops;

UPDATE laptops
SET cpu_name = SUBSTRING_INDEX(TRIM(cpu_name),' ',2);

SELECT DISTINCT cpu_name FROM laptops;

SELECT Memory FROM laptops;

ALTER TABLE laptops
ADD COLUMN memory_type VARCHAR(255) AFTER Memory,
ADD COLUMN primary_storage INTEGER AFTER memory_type,
ADD COLUMN secondary_storage INTEGER AFTER primary_storage;

SELECT Memory,
CASE
	WHEN Memory LIKE '%SSD%' AND Memory LIKE '%HDD%' THEN 'Hybrid'
    WHEN Memory LIKE '%SSD%' THEN 'SSD'
    WHEN Memory LIKE '%HDD%' THEN 'HDD'
    WHEN Memory LIKE '%Flash Storage%' THEN 'Flash Storage'
    WHEN Memory LIKE '%Hybrid%' THEN 'Hybrid'
    WHEN Memory LIKE '%Flash Storage%' AND Memory LIKE '%HDD%' THEN 'Hybrid'
    ELSE NULL
END AS 'memory_type'
FROM laptops;

UPDATE laptops
SET memory_type = CASE
	WHEN Memory LIKE '%SSD%' AND Memory LIKE '%HDD%' THEN 'Hybrid'
    WHEN Memory LIKE '%SSD%' THEN 'SSD'
    WHEN Memory LIKE '%HDD%' THEN 'HDD'
    WHEN Memory LIKE '%Flash Storage%' THEN 'Flash Storage'
    WHEN Memory LIKE '%Hybrid%' THEN 'Hybrid'
    WHEN Memory LIKE '%Flash Storage%' AND Memory LIKE '%HDD%' THEN 'Hybrid'
    ELSE NULL
END;

SELECT Memory,
REGEXP_SUBSTR(SUBSTRING_INDEX(Memory,'+',1),'[0-9]+'),
CASE WHEN Memory LIKE '%+%' THEN REGEXP_SUBSTR(SUBSTRING_INDEX(Memory,'+',-1),'[0-9]+') ELSE 0 END
FROM laptops;

UPDATE laptops
SET primary_storage = REGEXP_SUBSTR(SUBSTRING_INDEX(Memory,'+',1),'[0-9]+'),
secondary_storage = CASE WHEN Memory LIKE '%+%' THEN REGEXP_SUBSTR(SUBSTRING_INDEX(Memory,'+',-1),'[0-9]+') ELSE 0 END;

SELECT
primary_storage,
CASE WHEN primary_storage <= 2 THEN primary_storage*1024 ELSE primary_storage END,
secondary_storage,
CASE WHEN secondary_storage <= 2 THEN secondary_storage*1024 ELSE secondary_storage END
FROM laptops;

UPDATE laptops
SET primary_storage = CASE WHEN primary_storage <= 2 THEN primary_storage*1024 ELSE primary_storage END,
secondary_storage = CASE WHEN secondary_storage <= 2 THEN secondary_storage*1024 ELSE secondary_storage END;

SELECT * FROM laptops;

ALTER TABLE laptops DROP COLUMN gpu_name;

SELECT * FROM laptops;







EDA on cleaned data

In [ ]:
SELECT * FROM laptops;

-- head, tail and sample
SELECT * FROM laptops
ORDER BY `index` LIMIT 5;

SELECT * FROM laptops
ORDER BY `index` DESC LIMIT 5;

SELECT * FROM laptops
ORDER BY rand() LIMIT 5;

SELECT COUNT(Price) OVER(),
MIN(Price) OVER(),
MAX(Price) OVER(),
AVG(Price) OVER(),
STD(Price) OVER(),
PERCENTILE_CONT(0.25) WITHIN GROUP(ORDER BY Price) OVER() AS 'Q1',
PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY Price) OVER() AS 'Median',
PERCENTILE_CONT(0.75) WITHIN GROUP(ORDER BY Price) OVER() AS 'Q3'
FROM laptops
ORDER BY `index` LIMIT 1;

-- missing value
SELECT COUNT(Price)
FROM laptops
WHERE Price IS NULL;

-- outliers
SELECT * FROM (SELECT *,
PERCENTILE_CONT(0.25) WITHIN GROUP(ORDER BY Price) OVER() AS 'Q1',
PERCENTILE_CONT(0.75) WITHIN GROUP(ORDER BY Price) OVER() AS 'Q3'
FROM laptops) t
WHERE t.Price < t.Q1 - (1.5*(t.Q3 - t.Q1)) OR
t.Price > t.Q3 + (1.5*(t.Q3 - t.Q1));

SELECT t.buckets,REPEAT('*',COUNT(*)/5) FROM (SELECT price,
CASE
	WHEN price BETWEEN 0 AND 25000 THEN '0-25K'
    WHEN price BETWEEN 25001 AND 50000 THEN '25K-50K'
    WHEN price BETWEEN 50001 AND 75000 THEN '50K-75K'
    WHEN price BETWEEN 75001 AND 100000 THEN '75K-100K'
	ELSE '>100K'
END AS 'buckets'
FROM laptops) t
GROUP BY t.buckets;

SELECT Company,COUNT(Company) FROM laptops
GROUP BY Company;

SELECT cpu_speed,Price FROM laptops;

SELECT * FROM laptops;

SELECT Company,
SUM(CASE WHEN Touchscreen = 1 THEN 1 ELSE 0 END) AS 'Touchscreen_yes',
SUM(CASE WHEN Touchscreen = 0 THEN 1 ELSE 0 END) AS 'Touchscreen_no'
FROM laptops
GROUP BY Company;

SELECT DISTINCT cpu_brand FROM laptops;

SELECT Company,
SUM(CASE WHEN cpu_brand = 'Intel' THEN 1 ELSE 0 END) AS 'intel',
SUM(CASE WHEN cpu_brand = 'AMD' THEN 1 ELSE 0 END) AS 'amd',
SUM(CASE WHEN cpu_brand = 'Samsung' THEN 1 ELSE 0 END) AS 'samsung'
FROM laptops
GROUP BY Company;

-- Categorical Numerical Bivariate analysis
SELECT Company,MIN(price),
MAX(price),AVG(price),STD(price)
FROM laptops
GROUP BY Company;
-- Dealing with missing values
SELECT * FROM laptops
WHERE price IS NULL;
-- UPDATE laptops
-- SET price = NULL
-- WHERE `index` IN (7,869,1148,827,865,821,1056,1043,692,1114)

-- replace missing values with mean of price
UPDATE laptops
SET price = (SELECT AVG(price) FROM laptops)
WHERE price IS NULL;
-- replace missing values with mean price of corresponding company
UPDATE laptops l1
SET price = (SELECT AVG(price) FROM laptops l2 WHERE
			 l2.Company = l1.Company)
WHERE price IS NULL;

SELECT * FROM laptops
WHERE price IS NULL;
-- corresponsing company + processor
SELECT * FROM laptops;
-- Feature Engineering
ALTER TABLE laptops ADD COLUMN ppi INTEGER;

UPDATE laptops
SET ppi = ROUND(SQRT(resolution_width*resolution_width + resolution_height*resolution_height)/Inches);

SELECT * FROM laptops
ORDER BY ppi DESC;

ALTER TABLE laptops ADD COLUMN screen_size VARCHAR(255) AFTER Inches;

UPDATE laptops
SET screen_size =
CASE
	WHEN Inches < 14.0 THEN 'small'
    WHEN Inches >= 14.0 AND Inches < 17.0 THEN 'medium'
	ELSE 'large'
END;

SELECT screen_size,AVG(price) FROM laptops
GROUP BY screen_size;

-- One Hot Encoding

SELECT gpu_brand,
CASE WHEN gpu_brand = 'Intel' THEN 1 ELSE 0 END AS 'intel',
CASE WHEN gpu_brand = 'AMD' THEN 1 ELSE 0 END AS 'amd',
CASE WHEN gpu_brand = 'nvidia' THEN 1 ELSE 0 END AS 'nvidia',
CASE WHEN gpu_brand = 'arm' THEN 1 ELSE 0 END AS 'arm'
FROM laptops
